In [1]:
from selenium import webdriver
import time
import openpyxl
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup

# 최종 영화 csv 파일로 사용 / id number 있으면 그것도 불러옴
mlist = pd.read_excel('./추가재개봉리스트.xlsx')
indexnum = mlist['코드']
searching = mlist['영화명']
# rating = mlist['점수']

try:
    wb = openpyxl.load_workbook("KOBIS_재개봉추가.xlsx")
    sheet = wb.active
    print("불러오기 완료")
except:
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["인덱스", "검색어", "영화명", "영어명", "코드", "분류", "장르", "시간", "관람가", "국가",
                  "개봉일", "재개봉일", "개봉년도", "재개봉년도", "재개봉여부",
                  "총스크린수", "총상영횟수", "총누적매출액", "총누적관객수", "서울시스크린", "전국스크린", "서울시매출액", "전국매출액",
                  "서울시관객수", "전국관객수", "비고", "서울데이터"])
    print("새로운 파일을 만들었습니다.")

driver = webdriver.Chrome("./chromedriver")

for i in range(0, 17178):
    try:
        # 영화 검색
        driver.get("http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do")
        time.sleep(2)

        search_box = driver.find_element_by_css_selector("input#inp_solrSearch")
        search_box.send_keys(searching[i])
        search_button = driver.find_element_by_css_selector("button.btn_sch")
        search_button.click()

        time.sleep(3)

        # 검색 결과의 첫 번째 영화 클릭
        mname_button = driver.find_element_by_xpath("//a[@href='#none']")
        mname_button.click()
        time.sleep(2)

        #영화코드
        mcode = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[0].text.strip()
        # 검색한 영화와 실제 결과로 나온 영화가 일치하는지 알아보기 위해 수집
        movie_name = driver.find_element_by_css_selector("div.hd_layer strong.tit").text.strip()
        english_name = driver.find_element_by_css_selector("div.hd_layer > div").text.strip()
        # english_name = re.sub('[^a-zA-Z0-9]', '', mname)
        
        time.sleep(1)
        
        # 상세정보에서 개봉일/재개봉일 수집
        summary_info = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[3].text.strip()
        open_date = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[4].text.strip()

        try:
            reopen = driver.find_element_by_xpath("//dt[text()='재개봉일']")
            reopen_date = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[7].text.strip()
            re_whether = 1
        except:
            reopen_date = ""
            re_whether = 0

        stat_button = driver.find_element_by_xpath("//a[text()='통계정보']")
        stat_button.click()
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 총스크린수, 총상영횟수, 총누적매출액, 총누적관객수 구하기 - 상영타입별 누적통계에서
        try:
            num_list = soup.select("tr.summary > td.tar")
            total_screen = num_list[3].text.strip()
            total_show = num_list[4].text.strip()
            total_revenue = num_list[5].text.strip()
            total_attendance = num_list[6].text.strip()
        # 상영타입별 누적통계 없을 경우 - 0으로 채우기
        except:
            total_screen = 0
            total_show = 0
            total_revenue = 0
            total_attendance = 0

        # "서울시스크린", "서울시매출액", "서울시관객수", "전국스크린", "전국매출액", "전국관객수" 구하기 - 공식통계 정보란
        seoul_list = soup.select(
            ".cont_tab > div:nth-of-type(2) > div:nth-of-type(1) > table > tbody > tr > td[class=tar]")
        try:
            seoul_screen = seoul_list[0].text.strip()
            seoul_revenue = seoul_list[1].text.strip()
            seoul_attendance = seoul_list[2].text.strip()
        except:
            seoul_screen = 0
            seoul_revenue = 0
            seoul_attendance = 0

        try:
            whole_screen = seoul_list[3].text.strip()
            whole_revenue = seoul_list[4].text.strip()
            whole_attendance = seoul_list[5].text.strip()
        except:
            whole_screen = 0
            whole_revenue = 0
            whole_attendance = 0


        sheet.append([indexnum[i], searching[i], movie_name, english_name, mcode, summary_info, "", "", "", "",
                      open_date, reopen_date, "", "", re_whether,
                      total_screen, total_show, total_revenue, total_attendance, seoul_screen, whole_screen, seoul_revenue, whole_revenue,
                      seoul_attendance, whole_attendance, "", ""])
        print(movie_name, english_name)
        wb.save("KOBIS_재개봉추가.xlsx")

    except:
        print(indexnum[i], ',', searching[i], "오류 발생")
        continue

driver.close()
wb.save("KOBIS_재개봉추가.xlsx")

새로운 파일을 만들었습니다.
시네마 천국 시네마 천국 (Cinema Paradiso)
빽 투 더 퓨쳐 빽 투 더 퓨쳐 (Back To The Future)
10003 , 빽 투 더 퓨쳐 2 오류 발생
10007 , 스타워즈 에피소드 6 - 제다이의 귀환 오류 발생
10018 , 이티 오류 발생
10021 , 록키 오류 발생
10046 , 빠삐용 오류 발생
10071 , 대부 오류 발생
10086 , 미션 오류 발생
10100 , 탑건 오류 발생
10102 , 사운드 오브 뮤직 오류 발생
10110 , 영웅본색 오류 발생
10113 , 첩혈쌍웅 오류 발생
10114 , 아마데우스 오류 발생
10140 , 매드 맥스 오류 발생
10173 , 사랑과 영혼 오류 발생
10181 , 정무문 오류 발생
10218 , 로보캅 2 오류 발생
10250 , 해리가 샐리를 만났을 때 오류 발생
10262 , 델마와 루이스 오류 발생
10275 , 니키타 오류 발생
10314 , 마지막 황제 오류 발생
10453 , 졸업 오류 발생
10501 , 지옥의 묵시록 오류 발생
10526 , 블러드 심플 오류 발생
10561 , 대부 3 오류 발생
10670 , 천녀유혼 오류 발생
10686 , 천녀유혼 2 - 인간도 오류 발생
10741 , 연인 오류 발생
10874 , 베로니카의 이중 생활 오류 발생
10922 , 고령가 소년 살인사건 오류 발생
10936 , 가위손 오류 발생
10962 , 블레이드 러너 오류 발생
11031 , 열혈남아 오류 발생
11043 , 황비홍-천하무인 오류 발생
11104 , 동방불패 오류 발생
11167 , 가을 소나타 오류 발생
11290 , 그랑블루 오류 발생
11572 , 천녀유혼 3 - 도도도 오류 발생
11720 , 라스트 모히칸 오류 발생
11840 , 여왕 마고 오류 발생
12692 , 인도차이나 오류 발생
12787 , 베티블루 37.2 오류 발생
12888 , 미녀와 야수 오류 발생
12918 , 시애틀의 잠 못 이루는 밤 오류 발생
1

KeyError: 228